# Explicación de las notebooks y explicacón teorica de los Embeddings

# Explicación de la notebook 1

En este notebook se realiza la visualización de datos. Los datasets son los entregados por la catedra, que son los proporcionados por la competencia + atributos externos. Algunos de los atributos usa cantidad de clientes por dia, abierto en un dia o no, el clima del dia, viento, si es feriado o no, etc...

## Mética de la competencia ( llos scores son calculados con esta métrica)
$$
\textrm{RMSPE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} \left(\frac{\hat{y}_i - y_i}{y_i}\right)^2}
$$

donde:

- $y_i$ las ventas de un día particular de un store
- $\hat{y}_i$ ventas estimadas por el modelo
- $n$ es el número de predicciones realizadas

# Explicación de la notebook 2 y 3

En esta notebook y la 3 se hace el preprocesamiento de los datos y unión de todos los atributos en un unico archivo. Se buscan y solucionan casos de datos faltantes, se procesan las fechas para que esten en el formato correspondiente, se unen mas variables de googletrend. En el 3 se pasa stateholioday (que es categorica de 4 categorias) a booleana y se agrega información sobre distancias temporales a eventos para cada dia, como por ejemplo si un dia no es feriado, cuanto falta para el proximo y hace cuanto fue el ultimo.


# Explicación de la notebook 4

En esta notebook se separan las variables en categóricas y continuas. Las continuas, seran normalizadas mediante la resta de la media y división por el desvio para mejorar luego el entrenamiento. A las variables categóricas que no entén en formato numérico o booleano, se les realiza un "encoding" para pasarlas a numericas y poder utilizarlas para la regresión (se cambio la selección de categoricas y continuas dadas por default en la notebook y, con estos cambios, se consiguieron mejores resultados en las siguientes notebooks. Por ejemplo, "promo" o "Schoolholiday" esta seleccionada como continua y se le realizaba unan normalización antes pero solo toma valores de 0 o 1, por lo que se decidio tomarla como categorica). Además, como en las otras notebooks, se corrigio código que tiraba mensajes de error, como por ejemplo mas adelante que se usaba la variable StateHoliday_fw, que ya no existía porque se la reemplazo por StateHoliday_bool_fw.

# Explicación de la notebook 4.1 y 5

En estas notebook se generan los primeros modelos de embeddings sobre las variables categóricas y se muestra gráficamente el resultado de algunos de los embeddings. En esta notebook se relializa tambien una primera aproximación a la regresión.

Puede verse el resultado de embedding de Days of week y de Store gráficamente. Se puede ver que la relación de cercanía en los días es distinta entre sí, con dias mas alejados de otros. Esto destaca la utilidad de los embedding de encotrar y agregar relaciones entre los valores de las variables categoricas que en su espacio normal de representación no podían verse. Por ejemplo, domingo se encuentra muy alejada de las demas, probablemente porque las ventas un domingo son menores (o cero) a los otros días ya que posiblemente la tienda o no abra, o abra en horarios mas reducidos que un día normal. De esta manera se agregan relaciones entre los valores categoricos de las variables que agregan información a las mismas para auxiliar la regresión. Lo mismo para el caso de store, donde puede verse que, para las variables categoricas que indican el numero de tienda, que en priuncipio no tienen  relación entre si, al realizarse el embedding se ve en el espacio de embedding la relación geografica entre los stores.

# Explicación notebook 6

En esta notebook se realizó la regresión del modelo usando los embeddings sobre las variables categóricas. Se realizo la regresión mediante una red neuronal con función de output "linear". Se modificaron los embeddings para agregar las variables que se decidieron cambiar de continuas a categóricas en las notebooks anteriores. Se probó tambien realizar la regresión sin realizar este cambio de variables categoricas y continuas, dejando el default, y se lo comparo con la regresión realizada con las variables dispuestas por mi manualmente en categoricas y continuas. El resultado fue mejor de la segunda forma, pasando de score de 0.16028 a 0.12870 en Kaggler (esta comparación se realizó con los mismos hyperparametros, lr =0.01, first_hidden_units = 1000 ,second_hidden_units = 500, l2_lambda = 1e-3, 20 epochs y batchsize de 256).

### SUBMISSIONS

- Archivo de mejor resultado con hiperparametros optimos y cambiando categorias (categoricos y continuos): submision_mas_cat_True-False-linear-0.001-1000-20-256-0.01.csv (score = 0.12870)

parametros:  lr =0.01, first_hidden_units = 1000 ,second_hidden_units = 500, l2_lambda = 1e-3, 20 epochs y batchsize de 256

- Archivo de mejor resultado sin cambiar categorias con parametros optimos: submision_True-False-linear-0.001-1000-20-256-0.01.csv (score = 0.16028)

parametros: lr =0.01, first_hidden_units = 1000 ,second_hidden_units = 500, l2_lambda = 1e-3, 20 epochs y batchsize de 256:






# Explicación de la notebook 7

En esta notebook se utiliza el modelo de LIGHT GBM, un modelo basado en arboles de decisión. Primero se hizo una corrida con los parametros default del regresor excepto por algunos ingresados manualmente, que son n_estimators = 4000 learning_rate=0.05 y min_child_samples=5. Con estos parametros, se consiguio un score de 0.11412. Luego, se hizo un tunning de distintos hyperparametros, hasta llegar al mejor score, de 0.10715, que resulto también el mejor score de clasificación de todo el trabajo práctico. 

Para conseguir este score, se utilizaron los siguientes parametros:

params = {"objective" : "rmse",
          "boosting" : "gbdt", 
          "metric" : "rmse",
          "num_iterations" : 15148,
          "top_k" : 30, 
          "max_depth" : max_depth, 
          "num_leaves" : num_leaves, 
          "min_data_in_leaf" : min_data_in_leaf, 
          "learning_rate" : 0.02,
          "bagging_fraction" : 0.7, 
          "bagging_seed" : 3,
          "bagging_freq" : 5, 
          "feature_fraction" : 0.5, 
          "num_threads" : 4
         }
         
dataset_params = {"max_bin" : 200, 
                  "min_data_in_bin" : 3} 

Cabe destacar que se probó utilizar , con los parametros defalult (n_estimators = 4000 learning_rate=0.05 y min_child_samples=5), y se consiguio un resultado casi igual,al de sin embedding y parametros default, con score de 0.11555.

### SUBMISSIONS CON ESTOS SCORES: 

- Parametros default sin embedding: submisionn_True-5-4000-0.05.csv (score = 0.11412)

- Parametros default con embedding: submision_LGBM_embedd_True-5-4000-0.05-8-800-csv (Score = 0.11555)

- Parametros cambiados sin embedding: submision_LGBM_muchos_param_True-5-4000-0.02-8-800-.csv (score = 0.10715) EL MEJOR




